In [434]:
from scipy.io.wavfile import read
import numpy as np, ash
import soundfile as sf
import sounddevice as sd
import random

data, sr = sf.read('dog.wav', dtype='int16')  
print(sr)
sd.play(data)

44100


In [435]:
print(data)

[[-44 -37]
 [-45 -39]
 [-46 -43]
 ...
 [  0   1]
 [  0   2]
 [  0   0]]


In [436]:
import wave
import struct

def audio_to_binary(filename):
    with wave.open(filename, "rb") as wav_file:
        # Read the binary data from the wave file
        binary_data = wav_file.readframes(wav_file.getnframes())
        # Unpack the binary data and return it as a list of integers
        audio = list(struct.unpack("B"*len(binary_data), binary_data))
        return audio
        #return np.vectorize(np.binary_repr)(audio, 8)


def binary_to_audio(binary_data, filename, sample_rate=44100, sample_width=2, channels=2):
    # Open a wave file for writing
    with wave.open(filename, "w") as wav_file:
        # Set the sample width and number of channels
        wav_file.setsampwidth(sample_width)
        wav_file.setnchannels(channels)
        # Set the sample rate
        wav_file.setframerate(sample_rate)
        # Write the binary data to the wave file
        for byte in binary_data:
            data = struct.pack("B", byte)
            wav_file.writeframes(data)

In [437]:
audio = audio_to_binary("dog.wav")       #max = 255, min=0 list of integers
bin_audio_list =  np.vectorize(np.binary_repr)(audio, 8)

bin_audio_str = ''.join(bin_audio_list)


In [438]:
#binary_to_audio(audio, 'hello.wav')  

In [439]:
def DNA(line, l):
    Base_map = {"00":"A", "01":"G", "10":"C", "11":"T"}
    x=""
    temp = [line[i:i+2] for i in range(0, l, 2)]
    for j in range (l//2) :
         x += Base_map[temp[j]]
    
    return x

def revDNA(line, l):
    Code_map = {"A":"00", "G":"01", "C": "10", "T":"11"}
    x = ""
    temp = [line[i:i+1] for i in range(0,l)]
    for j in range(l):
        x += Code_map[temp[j]]
    return x

In [440]:
mapped_array = np.array([DNA(elem, 8) for elem in bin_audio_list])
print(mapped_array)

['TGGA' 'TTTT' 'TGCT' ... 'AAAA' 'AAAA' 'AAAA']


In [441]:
pre_encrypt = ''.join(mapped_array)


In [442]:
#print(pre_encrypt)


In [443]:
#key = np.array([random.randrange(2) for _ in range(200)])
key = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGT'
key_size = len(key)

In [444]:
print(revDNA(key, 47))

0011001110110110011100100111001001110010011100100111001001111101000010011100001001011100100111


In [445]:
bin_key = revDNA(key, 47)

y = int(bin_key[0:17],2) ^ int(bin_audio_str[0:17],2)
print('{0:b}'.format(y))

11100111010010011


In [446]:
a = 'ATATCTGCGTACGTACGTACGTACGTACGTTGAACGTAACGGTACGACCC'
la = len(a)
b = 'AGTCGT'
lb = len(b)
chunk = [a[0+i:lb+i] for i in range(0, la, lb)]
print("chunk = ", chunk)


chunk =  ['ATATCT', 'GCGTAC', 'GTACGT', 'ACGTAC', 'GTACGT', 'TGAACG', 'TAACGG', 'TACGAC', 'CC']


In [447]:
def Xor_string(a,b):
    x=''
    for i in range(len(a)):
        if (a[i] == b[i]):
            x+= '0'
        else:
            x+= '1'
    return x


In [448]:
key_val = revDNA(b, lb)
temp = chunk[len(chunk)-1]
x = Xor_string(key_val[0:len(temp)*2],revDNA(temp, len(temp)))
xorval = [Xor_string(key_val, revDNA(item,lb)) for item in chunk[0:len(chunk)-1]]
xorval.append(x)

print(xorval)

['001011011100', '011110010101', '011011000000', '001110010101', '011011000000', '110011101110', '110111000010', '110101110101', '1011']


In [449]:
def mapDNA(xorlist):
    j=0
    l = len(xorlist[0])
    x = []
    for xor in xorlist:
        litem = len(xor)
        for i in range(0, litem-1, 2):
            if(xor[i]=='1' and xor[i+1]=='1'):
                x.append((i+j*l)//2)
        j+=1
    return x


In [450]:
index = mapDNA(xorval)
print(index)


[2, 4, 7, 14, 19, 26, 30, 32, 34, 36, 38, 42, 45, 49]


In [451]:
randlen = [random.randint(1,6) for i in range (len(index))]
print(randlen)
    

[6, 5, 1, 4, 1, 4, 3, 1, 1, 2, 5, 3, 3, 5]


In [452]:
encr_str = []
x=''
for val in randlen:
    for i in range(val*2):
        x += str(random.randint(0,1))
    encr_str.append(DNA(x, len(x)))
    x=''

In [453]:
encr_str

['AGAGTC',
 'TGGTT',
 'T',
 'AAGC',
 'A',
 'TGAT',
 'GTT',
 'C',
 'T',
 'TG',
 'TGTTT',
 'AAG',
 'GCG',
 'TCCTC']

In [454]:
splitted=[a[0:index[0]+1]]
for i in range(len(index)-1): 
    splitted.append(a[index[i]+1:index[i+1]+1])
splitted.append(a[index[i+1]+1:la])

In [455]:
#a = 'ATA TC TGC GTACGTA CGTAC GTACGTA CGTT GA AC GT AA CGGT ACG ACT'
# index = [2, 4, 7, 14, 19, 26, 30, 32, 34, 36, 38, 42, 45, 48]
print(splitted)

['ATA', 'TC', 'TGC', 'GTACGTA', 'CGTAC', 'GTACGTA', 'CGTT', 'GA', 'AC', 'GT', 'AA', 'CGGT', 'ACG', 'ACCC', '']


In [456]:
#binary_to_audio(binary_data, "example3.wav")

In [457]:
ploughed=''
for i in range(len(encr_str)):
    ploughed += splitted[i]
    ploughed += encr_str[i]
ploughed += splitted[i+1]

In [458]:
print(splitted[i+1])

In [459]:
ploughed

'ATAAGAGTCTCTGGTTTGCTGTACGTAAAGCCGTACAGTACGTATGATCGTTGTTGACACTGTTGAATGTTTCGGTAAGACGGCGACCCTCCTC'

In [460]:
lp = len(ploughed)
ploughed= [ploughed[0+i:lb+i] for i in range(0, lp, lb)]

In [461]:
ploughed

['ATAAGA',
 'GTCTCT',
 'GGTTTG',
 'CTGTAC',
 'GTAAAG',
 'CCGTAC',
 'AGTACG',
 'TATGAT',
 'CGTTGT',
 'TGACAC',
 'TGTTGA',
 'ATGTTT',
 'CGGTAA',
 'GACGGC',
 'GACCCT',
 'CCTC']

In [462]:
ploughed_bin = [revDNA(item, len(item)) for item in ploughed]

In [463]:
ploughed_bin

['001100000100',
 '011110111011',
 '010111111101',
 '101101110010',
 '011100000001',
 '101001110010',
 '000111001001',
 '110011010011',
 '100111110111',
 '110100100010',
 '110111110100',
 '001101111111',
 '100101110000',
 '010010010110',
 '010010101011',
 '10101110']

In [464]:
ploughed_audio = [int(item, 2) for item in ploughed_bin]

In [465]:
ploughed_audio

[772,
 1979,
 1533,
 2930,
 1793,
 2674,
 457,
 3283,
 2551,
 3362,
 3572,
 895,
 2416,
 1174,
 1195,
 174]

In [466]:
binary_to_audio([item%256 for item in ploughed_audio], 'encrypted.wav')